In [75]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import os
import shutil
import cv2

from sklearn.model_selection import train_test_split
from PIL import Image


**DATA LOADING**

In [76]:
# make a dataframe from the csv file and pick product types that fall within the fashion mnist classes
df = pd.read_csv("styles.csv", usecols=[0, 4])
categorie_fashion_mnist = ["Tshirts", "Tops",
                            "Jeans", "Trousers", "Track Pants", "Leggings",
                            "Sweatshirts", "Sweaters",
                            "Dresses", "Kurtas", "Robe",
                            "Jackets", "Rain Jacket",
                            "Sandals", "Flip Flops", "Sports Sandals",
                            "Shirts",
                            "Casual Shoes", "Sports Shoes",
                            "Handbags",
                            "Formal Shoes"
                        ]

df_fashion_mnist = df[df.iloc[:, 1].isin(categorie_fashion_mnist)]

In [77]:
df_fashion_mnist['articleType'].unique()

array(['Shirts', 'Jeans', 'Track Pants', 'Tshirts', 'Casual Shoes',
       'Flip Flops', 'Handbags', 'Tops', 'Sandals', 'Sweatshirts',
       'Formal Shoes', 'Kurtas', 'Sports Shoes', 'Rain Jacket', 'Dresses',
       'Trousers', 'Jackets', 'Sports Sandals', 'Sweaters', 'Leggings',
       'Robe'], dtype=object)

In [78]:
# make a mapping to have the fashion mnist labels in place of their corresponding product types
mapping = {
    "T-shirt/top": ["Tshirts", "Tops"],
    "Trouser": ["Jeans", "Trousers", "Track Pants", "Leggings"],
    "Pullover": ["Sweatshirts", "Sweaters"],
    "Dress": ["Dresses", "Kurtas", "Robe"],
    "Coat": ["Jackets", "Rain Jacket"],
    "Sandal": ["Sandals", "Flip Flops", "Sports Sandals"],
    "Shirt": ["Shirts"],
    "Sneaker": ["Casual Shoes", "Sports Shoes"],
    "Bag": ["Handbags"],
    "Ankle boot": ["Formal Shoes"]
    }

# Reverse the mapping dictionary
reverse_mapping = {value: key for key, values in mapping.items() for value in values}
df_fashion_mnist['articleType'] = df_fashion_mnist['articleType'].replace(reverse_mapping)

C:\Users\Taher\AppData\Local\Temp\ipykernel_20924\3243347642.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fashion_mnist['articleType'] = df_fashion_mnist['articleType'].replace(reverse_mapping)


In [79]:
df_fashion_mnist


id  articleType
0      15970        Shirt
1      39386      Trouser
3      21379      Trouser
4      53759  T-shirt/top
5       1855  T-shirt/top
...      ...          ...
44439  12544  T-shirt/top
44440  42234  T-shirt/top
44441  17036      Sneaker
44442   6461       Sandal
44443  18842  T-shirt/top

[25977 rows x 2 columns]

In [80]:
# make a black background similar to the fashion mnist dataset and then convert the images to gray scale
images_folder  = "images"  
filtered_path  = "Fashion_Products_Images/data" 
rows_to_keep = []

if not os.path.exists(filtered_path):
    os.makedirs(filtered_path)

for index, row in df_fashion_mnist.iterrows():
    image_id = row['id']
    image_path_original = os.path.join(images_folder, str(image_id) + ".jpg")  
    
    
    if os.path.exists(image_path_original):
        
        image = cv2.imread(image_path_original)
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, background_mask = cv2.threshold(grayscale_image, 240, 255, cv2.THRESH_BINARY)
        object_mask = cv2.bitwise_not(background_mask)
        foreground = cv2.bitwise_and(image, image, mask=object_mask)
        black_background = np.zeros_like(image)
        image_with_black_background = cv2.add(black_background, foreground)
        final_image = cv2.cvtColor(image_with_black_background, cv2.COLOR_BGR2GRAY)
        
        image_path_grayscale = os.path.join(filtered_path, str(image_id) + ".jpg")  
        cv2.imwrite(image_path_grayscale, final_image)
        rows_to_keep.append(row)
    
df_final = pd.DataFrame(rows_to_keep)


In [81]:
# save the updated csv
df_final.to_csv("Fashion_Products_Images/fashion_proucts_filtered.csv", index=False)